# Quick Start

> Minimal Example of Hierarchical Reconciliation

Large collections of time series organized into structures at different aggregation levels often require their forecasts to follow their aggregation constraints, which poses the challenge of creating novel algorithms capable of coherent forecasts.

The `HierarchicalForecast` package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow classic hierarchical reconciliation.

In this notebook we will show how to use the `StatsForecast` library to produce base forecasts, and use `HierarchicalForecast` package to perform hierarchical reconciliation.

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismSmall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Libraries

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast datasetsforecast

## 2. Load Data

In this example we will use the `TourismSmall` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
import numpy as np
import pandas as pd

from datasetsforecast.hierarchical import HierarchicalData, HierarchicalInfo

In [ ]:
group_name = 'TourismSmall'
group = HierarchicalInfo.get_group(group_name)
Y_df, S_df, tags = HierarchicalData.load('./data', group_name)
S_df = S_df.reset_index(names="unique_id")
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
S_df.iloc[:6, :6]

,unique_id,nsw-hol-city,nsw-hol-noncity,vic-hol-city,vic-hol-noncity,qld-hol-city
0,total,1.0,1.0,1.0,1.0,1.0
1,hol,1.0,1.0,1.0,1.0,1.0
2,vfr,0.0,0.0,0.0,0.0,0.0
3,bus,0.0,0.0,0.0,0.0,0.0
4,oth,0.0,0.0,0.0,0.0,0.0
5,nsw-hol,1.0,1.0,0.0,0.0,0.0


In [ ]:
tags

{'Country': array(['total'], dtype=object),
 'Country/Purpose': array(['hol', 'vfr', 'bus', 'oth'], dtype=object),
 'Country/Purpose/State': array(['nsw-hol', 'vic-hol', 'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol',
        'nt-hol', 'nsw-vfr', 'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr',
        'tas-vfr', 'nt-vfr', 'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus',
        'wa-bus', 'tas-bus', 'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth',
        'sa-oth', 'wa-oth', 'tas-oth', 'nt-oth'], dtype=object),
 'Country/Purpose/State/CityNonCity': array(['nsw-hol-city', 'nsw-hol-noncity', 'vic-hol-city',
        'vic-hol-noncity', 'qld-hol-city', 'qld-hol-noncity',
        'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city', 'wa-hol-noncity',
        'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city', 'nt-hol-noncity',
        'nsw-vfr-city', 'nsw-vfr-noncity', 'vic-vfr-city',
        'vic-vfr-noncity', 'qld-vfr-city', 'qld-vfr-noncity',
        'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city', 'wa-vfr-noncity',
     

We split the dataframe in train/test splits.

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(group.horizon)
Y_train_df = Y_df.drop(Y_test_df.index)

## 3. Base forecasts

The following cell computes the *base forecast* for each time series using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

In [ ]:
fcst = StatsForecast(
    models=[AutoARIMA(season_length=group.seasonality), Naive()], 
    freq=group.freq, 
    n_jobs=-1
)
# Y_hat_df = fcst.forecast(df=Y_train_df, h=group.horizon, level=[80, 90])
Y_hat_df = fcst.forecast(df=Y_train_df, h=group.horizon)

c:\Users\ospra\miniconda3\envs\hierarchicalforecast-backup\lib\site-packages\utilsforecast\processing.py:384: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\ospra\miniconda3\envs\hierarchicalforecast-backup\lib\site-packages\utilsforecast\processing.py:440: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


## 4. Hierarchical reconciliation

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:

- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State', 
              top_down_method='forecast_proportions'),
    TopDown(method="average_proportions"),
    TopDown(method="proportion_averages"),
    MiddleOut(middle_level="Country/Purpose/State", top_down_method="average_proportions"),
    MiddleOut(middle_level="Country/Purpose/State", top_down_method="proportion_averages"),              
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, S=S_df, tags=tags)

## 5. Evaluation

The `HierarchicalForecast` package includes the `evaluate` function to evaluate the different hierarchies and we can use utilsforecast to compute the mean absolute error relative to a baseline model.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import mse

In [ ]:
df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])
evaluation = evaluate(df = df,
                      tags = tags,
                      train_df = Y_train_df,
                      metrics = [mse],
                      benchmark="Naive")

evaluation

# evaluation[["level", "metric", "AutoARIMA", "AutoARIMA/BottomUp", "AutoARIMA/TopDown_method-forecast_proportions"]]

,level,metric,AutoARIMA,Naive,AutoARIMA/BottomUp,Naive/BottomUp,AutoARIMA/TopDown_method-forecast_proportions,Naive/TopDown_method-forecast_proportions,AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-forecast_proportions,Naive/MiddleOut_middle_level-Country/Purpose/State_top_down_method-forecast_proportions,AutoARIMA/TopDown_method-average_proportions,Naive/TopDown_method-average_proportions,AutoARIMA/TopDown_method-proportion_averages,Naive/TopDown_method-proportion_averages,AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-average_proportions,Naive/MiddleOut_middle_level-Country/Purpose/State_top_down_method-average_proportions,AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-proportion_averages,Naive/MiddleOut_middle_level-Country/Purpose/State_top_down_method-proportion_averages
0,Country,mse-scaled,0.317897,1.0,0.367078,1.0,0.317897,1.0,0.305053,1.0,0.317897,1.000000,0.317897,1.000000,0.305053,1.000000,0.305053,1.000000
1,Country/Purpose,mse-scaled,0.318950,1.0,0.233606,1.0,0.262216,1.0,0.196062,1.0,0.317642,0.690007,0.320225,0.688443,0.196062,1.000000,0.196062,1.000000
2,Country/Purpose/State,mse-scaled,0.268057,1.0,0.281189,1.0,0.320349,1.0,0.268057,1.0,0.507569,0.753117,0.511356,0.755557,0.268057,1.000000,0.268057,1.000000
3,Country/Purpose/State/CityNonCity,mse-scaled,0.292136,1.0,0.292136,1.0,0.323261,1.0,0.281041,1.0,0.504631,0.723686,0.509784,0.727194,0.279817,0.915583,0.280599,0.916201
4,Overall,mse-scaled,0.308942,1.0,0.295690,1.0,0.297072,1.0,0.255085,1.0,0.362721,0.817154,0.364775,0.817238,0.254956,0.991166,0.255038,0.991231


### References
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773(787)](http://www.jstor.org/stable/1815532).
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)